In [ ]:
import cv2
import dlib

In [ ]:
print('cv2 version:', cv2.__version__)
print('dlib version:',dlib.__version__)

In [ ]:
# For using CPU only, I modified the routine to capture video into images first
# and batch process the images later, which takes a while to complete without a GPU

max_image = 100

# you need save_dir pre-created
save_dir = 'save_image'

# increase this number to capture more 'different' frames
skip_frames=45

count = skip_frames;
index = 1

video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    
    if count >= 0:
        ret, frame = video_capture.read()
        cv2.imshow('frame',frame)
        cv2.waitKey(1)
        count -= 1
    
    count = skip_frames
    file_name = save_dir + '/' + 'frame_' + str(index) + '.jpg';
    
    # write
    print('writing frame:', index)
    cv2.imwrite(file_name , frame)
    
    index += 1
    if index > max_image:
        break

video_capture.release()
cv2.destroyAllWindows()

In [ ]:
# you need a dlib compiled with GPU to work well

shape_file = 'shape_predictor_68_face_landmarks.dat'
feature_detector = predictor = dlib.shape_predictor(shape_file)

face_file = 'mmod_human_face_detector.dat'
face_detector = dlib.cnn_face_detection_model_v1(face_file)

In [ ]:
## some videowriter props

cap = cv2.VideoCapture(0)

sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

cap.release()
cv2.destroyAllWindows()

# increase this number for smoother mp4 play.
fps = 5

#fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
#fourcc = cv2.VideoWriter_fourcc('m', 'p', 'e', 'g')
fourcc = cv2.VideoWriter_fourcc(*'mpeg')

## open and set props
vout = cv2.VideoWriter()
vout.open('output.mp4',fourcc,fps,sz,True)


for i in range(max_image):
    file_name = save_dir + '/' + 'frame_' + str(i+1) + '.jpg';
    
    print('detecting face for image:', file_name)
    
    frame = cv2.imread(file_name,cv2.IMREAD_COLOR )
    
    # convert to rgb for dlib
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # detect faces
    dets = face_detector(img, 1)
    
    # rectangles around faces
    for i, d in enumerate(dets):
        x = d.rect.left()
        y = d.rect.top()
        x2 = d.rect.right()
        y2 = d.rect.bottom()
        
        cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
        
        # detect features
        coor = feature_detector(img , d.rect)
        if not coor is None:
            for f in range(0,68):
                cv2.circle(frame , (coor.part(f).x , coor.part(f).y) , 1, (255,255,255),1 )
    
    vout.write(frame)
    
vout.release()